In [ ]:
from typing import Dict, Optional
from sortedcontainers import SortedDict  # type: ignore
import os

import ipysheet as sheet
from IPython.display import display, HTML
import mammoth

from const import IDX_COL, STYLE_COL
from config import FROM_LANG, TO_LANG
from setup import sl_sem, gr_sem

from util import ord_word
from semantics import TableSemantics
from hiliting import _hilited_col

from merger import merge
from aggregator import aggregate
from exporter import export_docx
from generator import generate_docx

In [ ]:
sla = SortedDict(ord_word)
gre = SortedDict(ord_word)

pairs = [
    TableSemantics(sl_sem, gr_sem, label="от славянски основен към гръцки", result=sla),
    TableSemantics(
        sl_sem.var, gr_sem, label="от славянски вариант към гръцки", result=sla
    ),
    TableSemantics(gr_sem, sl_sem, label="от гръцки основен към славянски", result=gre),
    TableSemantics(
        gr_sem.var, sl_sem, label="от гръцки вариант към славянски", result=gre
    ),
]
sem = pairs[0]

In [ ]:
def _style2str(s: Dict[str, str], bgs: Dict[str, Optional[str]]) -> str:
    """take font style from index and presence of background in selected named columns"""
    result = [k for k, v in bgs.items() if v]
    if s and s["fontWeight"] == "bold":
        result.append("bold")
    if s and s["fontStyle"] == "italic":
        result.append("italic")
    return "|".join(result)


def hilite(row, c):
    color = _hilited_col(row, c)
    if color:
        return f"#{color[2:]}" if len(color) == 8 else f"#{color}"
    return None


def bold(c, style=None):
    if c != IDX_COL:
        return None
    if not style:
        return None
    if "bold" in style:
        return "bold"
    return None


def italic(c, style=None):
    if c != IDX_COL:
        return None
    if not style:
        return None
    if "italic" in style:
        return "italic"
    return None

In [ ]:
rows = [
    ["бꙑше H", "бꙑт\ue205", "", "gramm.", "25/123b05"]
    + ["om.", ""] * 2
    + [""] * 3
    + ["gramm."]
    + [""] * 13
    + ["hl00:FFF8CBAD|hl03:FFBDD7EE"]
    + ["1"] * 4,
    ["рекл\ue205 H"]
    + [""] * 3
    + ["25/123b05", "рѣша", "ко г\ue010лще рѣша• ꙗ-", "рещ\ue205"]
    + [""] * 3
    + ["φασὶν", "φημί"]
    + [""] * 13
    + ["hl00:FFF8CBAD"]
    + ["1"] * 4,
]


input = sheet.sheet(rows=len(rows) + 1, columns=STYLE_COL)
# styled = sheet.cell(1, 9, background_color="lightblue")
# rows = sheet.cell_range(rows)
cells = [
    [
        sheet.cell(
            r,
            c,
            rows[r][c],
            background_color=hilite(vr, c),
            font_style=italic(c, vr[STYLE_COL]),
            font_weight=bold(c, vr[STYLE_COL]),
        )
        for c, vc in enumerate(vr[:STYLE_COL])
    ]
    for r, vr in enumerate(rows)
]
input

# Preprocess lines

In [ ]:
lines = []
blank = False
for row in cells:
    line = [cell.value.strip() if cell.value else cell.value for cell in row]
    # Two consequent blank lines
    if blank and not [l for l in line if l]:
        break
    irow = [v for v in row]
    bgs = {
        f"hl{v:02d}": irow[v].style["backgroundColor"]
        for v in sem.cols()
        if "backgroundColor" in irow[v].style
    }
    line.append(_style2str(row[IDX_COL].style, bgs))
    lines.append(line)
    blank = not [l for l in line if l]
print(f"{len(lines)} думи")

In [ ]:
print(lines)

In [ ]:
for p in pairs:
    print(f"Събиране на многоредови преводи {p.label}...")
    merged = merge(lines, p.orig, p.trans)
    print(f"{len(merged)} думи")

    print(f"Кондензиране {p.label}...")
    before = len(p.result)
    p.result = aggregate(merged, p.orig, p.trans, p.result)
    after = len(p.result)
    print(f"{after-before} леми")

In [ ]:
export_fname = "temp.docx"

# Експорт славянски...

In [ ]:
export_docx(sla, FROM_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages)  # Any messages, such as warnings during conversion
    display(HTML(result.value))

# Експорт гръцки...

In [ ]:
export_docx(gre, TO_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages)  # Any messages, such as warnings during conversion
    display(HTML(result.value))

# Генериране славянски...

In [ ]:
generate_docx(sla, FROM_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages)  # Any messages, such as warnings during conversion
    display(HTML(result.value))

# Генериране гръцки...

In [ ]:
generate_docx(gre, TO_LANG, export_fname)
with open(export_fname, "rb") as docx_file:
    result = mammoth.convert_to_html(docx_file)
    print(result.messages)  # Any messages, such as warnings during conversion
    display(HTML(result.value))

In [ ]:
os.remove(export_fname)